## 🌍 Script Purpose: Download & Prepare Basemap Layers from Natural Earth

This script automates the download, extraction, and conversion of selected **Natural Earth** vector layers at your preferred generalization (10m, 50m, or 110m) and exports them as GeoPackage (`.gpkg`) files, ready for use in GIS or mapping projects.

**Key features:**
- Download only the layers you need (countries, rivers, coastlines, etc.)
- Select your preferred data resolution (detail level)
- Everything is saved in a clean output folder, only the final `.gpkg` files are kept
- No manual downloading or file management needed

**Typical use cases:**
- Quickly assemble background layers for cartography or spatial analysis
- Prepare clean and up-to-date base layers for QGIS, Python, R, or web mapping workflows

➡️ **Just set your parameters in the `USER SETTINGS` block below, and run the script!**



---
## 🛠️ User Settings: Basemap Layers Download

In [12]:
# ========== USER SETTINGS ==========

# 🌍 Choose the data resolution/generalization ("10m", "50m", or "110m")
RESOLUTION = "10m"

# 📚 List of layers to download (pick from NE_LAYERS_AVAILABLE below)
LAYERS_TO_DOWNLOAD = [
    ("cultural", "admin_0_countries"),
    ("physical", "ocean"),
    ("physical", "rivers_lake_centerlines"),
    ("cultural", "populated_places"),
    ("cultural", "roads"),
    ("physical", "coastline"),
]

# 📁 Output directory for the .gpkg files
OUTPUT_DIR = "data_input/basemap_elements"

# ========== END USER SETTINGS ==========

# List of layers

In [ ]:
# List of all available Natural Earth layers (exhaustive, by category)
NE_LAYERS_AVAILABLE = [
    # --- Cultural layers ---
    ("cultural", "admin_0_antarctic_claim_limits"),
    ("cultural", "admin_0_boundary_lines_land"),
    ("cultural", "admin_0_countries"),
    ("cultural", "admin_0_countries_lakes"),
    ("cultural", "admin_0_disputed_areas"),
    ("cultural", "admin_0_map_units"),
    ("cultural", "admin_0_pacific_groupings"),
    ("cultural", "admin_0_seams"),
    ("cultural", "admin_0_sovereignty"),
    ("cultural", "admin_1_states_provinces"),
    ("cultural", "admin_1_states_provinces_lakes"),
    ("cultural", "admin_1_states_provinces_lines"),
    ("cultural", "airports"),
    ("cultural", "built_up_areas"),
    ("cultural", "land_use_a"),
    ("cultural", "parks_and_protected_lands_area"),
    ("cultural", "parks_and_protected_lands_line"),
    ("cultural", "parks_and_protected_lands_point"),
    ("cultural", "populated_places"),
    ("cultural", "populated_places_simple"),
    ("cultural", "railroads"),
    ("cultural", "roads"),
    ("cultural", "roads_north_america"),
    ("cultural", "urban_areas"),
    ("cultural", "urban_areas_landscan"),
    ("cultural", "water_areas"),
    ("cultural", "water_lines"),
    # --- Physical layers ---
    ("physical", "antarctic_ice_shelves_lines"),
    ("physical", "antarctic_ice_shelves_polys"),
    ("physical", "coastline"),
    ("physical", "geographic_lines"),
    ("physical", "glaciated_areas"),
    ("physical", "graticules_1"),
    ("physical", "graticules_5"),
    ("physical", "graticules_10"),
    ("physical", "lakes"),
    ("physical", "lakes_europe"),
    ("physical", "land"),
    ("physical", "minor_islands"),
    ("physical", "ocean"),
    ("physical", "playas"),
    ("physical", "reefs"),
    ("physical", "rivers_europe"),
    ("physical", "rivers_lake_centerlines"),
    ("physical", "rivers_north_america"),
    ("physical", "tectonic_plates"),
]


# Function

In [7]:
import os
import requests
import shutil
from zipfile import ZipFile
from pathlib import Path
import geopandas as gpd

def download_and_convert_natural_earth(layers, resolution="10m", output_dir="data_input/basemap_elements"):
    """
    Télécharge, charge, exporte en GPKG, et nettoie pour une liste de couches Natural Earth.
    On ne garde que le .gpkg final.
    """
    os.makedirs(output_dir, exist_ok=True)
    base_url_tpl = "https://naturalearth.s3.amazonaws.com/{res}_{cat}/ne_{res}_{name}.zip"

    for category, name in layers:
        url = base_url_tpl.format(res=resolution, cat=category, name=name)
        zip_path = Path(output_dir) / f"ne_{resolution}_{name}.zip"
        extract_path = Path(output_dir) / f"ne_{resolution}_{name}"
        gpkg_path = Path(output_dir) / f"ne_{resolution}_{name}.gpkg"

        print(f"\n🔻 Traitement: {name} ({category}, {resolution})")
        
        # 1. Télécharger si pas déjà là
        if not zip_path.exists():
            r = requests.get(url)
            if r.status_code != 200:
                print(f"  ❌ Erreur: {r.status_code} pour {url}")
                continue
            with open(zip_path, "wb") as f:
                f.write(r.content)
            print("  ✅ ZIP téléchargé.")
        else:
            print("  ⚡ ZIP déjà présent.")

        # 2. Dézipper
        if not extract_path.exists():
            with ZipFile(zip_path, "r") as zip_ref:
                zip_ref.extractall(extract_path)
            print("  ✅ Dézippé.")
        else:
            print("  ⚡ Dossier déjà extrait.")

        # 3. Trouver le .shp principal
        shp_files = list(extract_path.glob("*.shp"))
        if not shp_files:
            print("  ❌ Pas de fichier .shp trouvé !")
            continue
        shp_path = shp_files[0]

        # 4. Charger dans GeoPandas
        try:
            gdf = gpd.read_file(shp_path)
            print("  ✅ Chargé avec GeoPandas.")
        except Exception as e:
            print(f"  ❌ Erreur lecture GeoPandas: {e}")
            continue

        # 5. Exporter en GPKG
        gdf.to_file(gpkg_path, driver="GPKG")
        print(f"  ✅ Exporté en GPKG : {gpkg_path}")

        # 6. Nettoyer : supprimer le ZIP, le dossier extrait
        try:
            os.remove(zip_path)
            print("  🗑️ ZIP supprimé.")
        except Exception:
            pass
        try:
            shutil.rmtree(extract_path)
            print("  🗑️ Dossier extrait supprimé.")
        except Exception:
            pass

    print("\n✨ Terminé ! Tous les fichiers .gpkg sont dans :", output_dir)

# Function call

In [ ]:
# --- Validation (optional but recommended)
def validate_layers(layers, available_layers):
    available_set = set(available_layers)
    for l in layers:
        if l not in available_set:
            raise ValueError(f"Layer not found: {l}")

validate_layers(LAYERS_TO_DOWNLOAD, NE_LAYERS_AVAILABLE)
if RESOLUTION not in ["10m", "50m", "110m"]:
    raise ValueError(f"Invalid resolution: {RESOLUTION}")

# --- Call your function (assuming download_and_convert_natural_earth is defined above)
download_and_convert_natural_earth(
    LAYERS_TO_DOWNLOAD,
    resolution=RESOLUTION,
    output_dir=OUTPUT_DIR
)


🔻 Traitement: admin_0_countries (cultural, 110m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_110m_admin_0_countries.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: ocean (physical, 110m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_110m_ocean.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: rivers_lake_centerlines (physical, 110m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_110m_rivers_lake_centerlines.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: populated_places (cultural, 110m)
  ✅ ZIP téléchargé.
  ✅ Dézippé.
  ✅ Chargé avec GeoPandas.
  ✅ Exporté en GPKG : data_input/basemap_elements/ne_110m_populated_places.gpkg
  🗑️ ZIP supprimé.
  🗑️ Dossier extrait supprimé.

🔻 Traitement: roads (cultural, 110m)
  ❌ Erreur: 